In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
df1 = pd.read_csv('archive/lap_times.csv')
df2 = pd.read_csv('archive/races.csv')

In [ ]:
#Eldobjuk a position és az ido oszlopot, és millisecond-al dolgozunk
df1 = df1.drop('time', axis=1)
df1 = df1.drop('position', axis=1)
#Eldobjuk a 2010 előtti versenyeket, mivel 2010be tiltották meg a tankolást,
#és az üzemanyag súlya nagyban befolyásolja a köridőt, és így körről körre gyorsabb is lesz az autó, "tanulhatóbb"
df2 = df2[df2['year'] > 2009]
#eldobunk további felesleges oszlopokat
df2 = df2.drop(['round','round','date','time','url','fp1_date','fp1_time','fp2_date','fp2_time','fp3_date','fp3_time','quali_date','quali_time', 'sprint_date', 'sprint_time'], axis=1)

In [ ]:
#Az inner-el csak azokat merge-eljük amiknek mind2 táblában van raceId
merged_df = pd.merge(df1, df2, on='raceId', how='inner')

In [ ]:
#Leszűkítjük a keresést Hamiltonra, hogy egyszerűbb legyen (már sok driver nincs is a formula 1ben)
Ham = merged_df[(merged_df['driverId'] == 1)]

In [ ]:
median = Ham['milliseconds'].median()
print(median)
Ham['milliseconds'] = np.where(Ham['milliseconds'] > median - 1000, median - 2000, Ham['milliseconds'])

#Kiválasztjuk Monza-t
Ham = Ham[(Ham['circuitId'] == 14)]

In [ ]:
#Adatok előkészítése
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(Ham[['lap', 'milliseconds', 'year']])

lookback = 5  #Az előző 5 kör adatait használjuk a következő köridő előrejelzéséhez
X = []
Y = []
for i in range(lookback, len(scaled_data)):
    X.append(scaled_data[i-lookback:i])
    Y.append(scaled_data[i, 1])  #milliseconds oszlop (köridő) a célváltozó

X = np.array(X)
Y = np.array(Y)

#Adatok felosztása tanító és teszt adathalmazra
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Modell létrehozása és tanítása
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(lookback, 3)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=16)

# Előrejelzés tesztadatokra
predictions = model.predict(X_test)

# Eredmények visszaskálázása
y_test = y_test.reshape(-1, 1)
scaled_predictions = np.concatenate((X_test[:, -1, :2], predictions), axis=1)
predictions = scaler.inverse_transform(scaled_predictions)[:, -1]

# Eredmények kiértékelése
mse = np.mean((predictions - y_test[:, 1])**2)
print("Mean Squared Error:", mse)

# Eredmények megjelenítése

plt.plot(y_test[:, 1], label='Valós köridő')
plt.plot(predictions, label='Előrejelzett köridő')
plt.xlabel('Körök')
plt.ylabel('Köridő (milliszekundom)')
plt.title('Formula 1 Köridő Előrejelzés')
plt.legend()
plt.show()